In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation as animation
import glob
import ipywidgets as widgets
import h5py
from IPython.display import HTML
import tqdm
from tslearn.clustering import TimeSeriesKMeans

In [ ]:
window = 150

In [ ]:
main_df = pd.read_pickle("/data/longterm/10/athira/July17_features_combined_noLightStimuli.pickle")
motif_df = pd.read_hdf(f"correct_frame_motifs_{window}_nclusters_15_meanvarnorm_rollingmean.hdf5")
files = motif_df.filename.unique()
raw_skel_files = glob.glob("/data/longterm/10/athira/Tierpsy_skeleton_files_July13/*skeletons.hdf5")
model = TimeSeriesKMeans.from_json(f"model_with_context_n15_w{window}.json")


In [ ]:
motif_df["labels"] = model.labels_

In [ ]:
def read_skeleton_as_array(file_path):
    file_path = list(filter(lambda x: file_path in x, raw_skel_files))[0]
    try:
        skeletons_store = h5py.File(file_path, 'r')
        skeletons = np.array(skeletons_store['coordinates']['skeletons'])

    except IOError:
        print("Unable to read the file : check if the file exists")

    except KeyError:
        skeletons = np.array(skeletons_store['skeleton'])

    return skeletons

def rotate(x, y, angle):
    xr = x*np.cos(angle) - y* np.sin(angle)
    yr = x*np.sin(angle) + y* np.cos(angle)
    return xr,yr
def get_base_angle(x,y, degrees = False):
    angle = np.arctan(y/x)
    if y < 0:
        angle *=-1
    if not degrees:
        return(angle)
    else:
        return(np.degrees(angle))
def get_skeleton_motif(motif):
    skel = read_skeleton_as_array(motif.filename)
    start = motif.frame
    stop = motif.frame + window
    skel =  skel[start:stop, :,:]
    return(skel)

In [ ]:
plt.close("all")

In [ ]:
import seaborn as sns

In [ ]:
palette = sns.color_palette("nipy_spectral", n_colors=window)

In [ ]:

# def make_animation(cluster):
#     def animate(i):
#         for j, m in enumerate(selected_motifs):
#             if i%150 == 0:
#                 axes.ravel()[j].clear()
#             frame = m[i,:,:]
#             xs = frame[:,0]
#             ys = frame[:,1]
#             angle = get_base_angle(xs[12], ys[12]) - np.pi
# #             xs = xs-frame[12, 0]
# #             ys = ys-frame[12, 1]
# #             xs, ys = rotate(xs, ys, angle)
#             axes.ravel()[j].plot(xs, ys, c = palette[i], alpha = 0.4, lw = 0.5)
#             axes.ravel()[j].axis("off")
#             axes.ravel()[j].axis("equal")
        
#     twelves = motif_df[motif_df["labels"] == cluster]
#     gets = np.random.choice(np.arange(0,len(twelves),1), 225)
#     selected_motifs = [get_skeleton_motif(twelves.iloc[get]) for get in gets]
#     fig, axes = plt.subplots(15,15, figsize = (10,10))
#     anim = animation(fig, animate, frames = 150, interval = 100)
#     anim.save(f"gifs_{window}_neck/anim_{str(cluster).zfill(2)}_COLOR_unaligned.gif", writer = "imagemagick", dpi = 300)
# #     anim = animation(fig, animate, frames = 30, interval = 100).to_jshtml()
# #     display(HTML(anim))

In [ ]:
def make_plot(cluster):
    twelves = motif_df[motif_df["labels"] == cluster]
    gets = np.random.choice(np.arange(0,len(twelves),1), 225, replace = False)
    selected_motifs = [get_skeleton_motif(twelves.iloc[get]) for get in gets]
    fig, axes = plt.subplots(15,15, figsize = (10,10), dpi = 1200)
    for j, m in enumerate(selected_motifs):
        for i in range(window):
            frame = m[i,:,:]
            xs = frame[:,0]
            ys = frame[:,1]
            angle = get_base_angle(xs[12], ys[12]) - np.pi
    #             xs = xs-frame[12, 0]
    #             ys = ys-frame[12, 1]
    #             xs, ys = rotate(xs, ys, angle)
            axes.ravel()[j].plot(xs, ys, c = palette[i], alpha = 0.4, lw = 0.5)
            axes.ravel()[j].axis("off")
            axes.ravel()[j].axis("equal")
    return fig, axes, twelves.iloc[gets]
        


In [ ]:
import os

In [ ]:
savepath = "high_dpi_coloured_skeletons_with_preserved_info"
if not os.path.exists(savepath):
    os.mkdir(savepath)
for c in tqdm.tqdm(range(15)):
    clusterpath = os.path.join(savepath, f"window_{window}_cluster_{str(c).zfill(2)}")
    if not os.path.exists(clusterpath):
        os.mkdir(clusterpath)
    fig, axes, selected_motifs = make_plot(c)
    axes = axes.ravel()
    fig.savefig(f"{clusterpath}/{window}_{c}.svg")
    fig.savefig(f"{clusterpath}/{window}_{c}.png")
    for i, ax in enumerate(axes):
        axes[i].set_title(str(i+1))
    fig.savefig(f"{clusterpath}/{window}_{c}_numbered.svg")
    fig.savefig(f"{clusterpath}/{window}_{c}_numbered.png")
    plt.close("fig")
    selected_motifs = selected_motifs.drop("motifs", axis = 1)
    selected_motifs["plot_number"] = np.arange(1,226)
    selected_motifs.to_csv(f"{clusterpath}/meta_cluster_{str(c).zfill(2)}", sep = "\t")

In [ ]:
axes

In [ ]:
selected_motifs

In [ ]:
for c in tqdm.tqdm(range(15)):
    try:
        make_animation(c)
    except Exception as e:
        print(c, str(e))

In [ ]:
plt.close("all")

In [ ]:
window = 30

palette = sns.color_palette("nipy_spectral", n_colors=window)

main_df = pd.read_pickle("/data/longterm/10/athira/July17_features_combined_noLightStimuli.pickle")
motif_df = pd.read_hdf(f"correct_frame_motifs_{window}_nclusters_15_meanvarnorm_rollingmean.hdf5")
files = motif_df.filename.unique()
raw_skel_files = glob.glob("/data/longterm/10/athira/Tierpsy_skeleton_files_July13/*skeletons.hdf5")
model = TimeSeriesKMeans.from_json(f"model_with_context_n15_w{window}.json")

motif_df["labels"] = model.labels_

def get_skeleton_motif(motif):
    skel = read_skeleton_as_array(motif.filename)
    start = motif.frame
    stop = motif.frame + window
    skel =  skel[start:stop, :,:]
    return(skel)

def make_animation(cluster):
    def animate(i):
        for j, m in enumerate(selected_motifs):
            if i%30 == 0:
                axes.ravel()[j].clear()
            frame = m[i,:,:]
            xs = frame[:,0]
            ys = frame[:,1]
            angle = get_base_angle(xs[12], ys[12]) - np.pi
#             xs = xs-frame[12, 0]
#             ys = ys-frame[12, 1]
#             xs, ys = rotate(xs, ys, angle)
            axes.ravel()[j].plot(xs, ys, c = palette[i], alpha = 0.4, lw = 0.5)
            axes.ravel()[j].axis("off")
            axes.ravel()[j].axis("equal")
        
    twelves = motif_df[motif_df["labels"] == cluster]
    gets = np.random.choice(np.arange(0,len(twelves),1), 225)
    selected_motifs = [get_skeleton_motif(twelves.iloc[get]) for get in gets]
    fig, axes = plt.subplots(15,15, figsize = (10,10))
    anim = animation(fig, animate, frames = 30, interval = 100)
    anim.save(f"gifs_{window}_neck/anim_{str(cluster).zfill(2)}_COLOR_unaligned.gif", writer = "imagemagick")
    
for c in tqdm.tqdm(range(15)):
    try:
        make_animation(c)
    except Exception as e:
        print(c, str(e))


In [ ]:
def translate(skel_coords, neck_ind):
    
    diff = skel_coords[neck_ind]
    skel_coords_trans = skel_coords - diff
    
    return skel_coords_trans

def angle(x,y, degrees = False):
    angle = np.arctan2(y,x)
#     if y < 0:
#         angle *=-1
    if not degrees:
        return(angle)
    else:
        return(np.degrees(angle))

def rotate_matrix(skel_array,angle):
    xr = skel_array[:,0]*np.cos(angle) - skel_array[:,1]* np.sin(angle)
    yr = skel_array[:,0]*np.sin(angle) + skel_array[:,1]* np.cos(angle)
    return xr,yr

def rotate(x, y, angle):
    xr = x*np.cos(angle) - y* np.sin(angle)
    yr = x*np.sin(angle) + y* np.cos(angle)
    return xr,yr

In [ ]:
np.vstack(random_selection).shape

In [ ]:
random_skels

In [ ]:

fig, ax = plt.subplots(3,5, figsize = (15,5))
ax = ax.ravel()
for cluster in range(15):
    motifs = motif_df[motif_df["labels"] == cluster]
    random_selection = [get_skeleton_motif(motifs.iloc[i]) for i in np.random.choice(range(len(motifs)), 1500)]



    ax[cluster].axis("equal")
    ax[cluster].axis("off")
#     xs_base = random_selection[0][0,:,0]
#     ys_base = random_selection[0][0,:,1]
#     angle = get_base_angle(xs_base[-1]-xs_base[0], ys_base[-1]-ys_base[0]) - np.pi
    skels = np.vstack(random_selection)
    random_indexes = np.random.choice(range(len(skels)), 100)
    for index in random_indexes:
        s = skels[index]
        
        s = translate(s, 12)
        rot_angle = angle(s[-1,0], s[-1,1], degrees = False)

        if rot_angle > np.pi/2:
            rotation = 3*(np.pi/2)-rot_angle
        else:
            rotation = -(np.pi/2+rot_angle)

        xs,ys = rotate_matrix(s, rotation)
        
        if not np.min(ys) < -100:
            ax[cluster].plot(xs,ys, color = "b", alpha = 0.1)
            ax[cluster].set_title(cluster)
            
# fig.savefig(f"motifs_static_{window}.svg")
# fig.savefig(f"motifs_static_{window}.png")
        

In [ ]:
import seaborn as sns

In [ ]:
palette = sns.color_palette("nipy_spectral", n_colors=window)

In [ ]:
palette

In [ ]:
window

In [ ]:
import os
os.mkdir(f"prog_col_skels_{window}")

In [ ]:

for cluster in range(15):
    motifs = motif_df[motif_df["labels"] == cluster]
    skels = random_selection = [get_skeleton_motif(motifs.iloc[i]) for i in np.random.choice(range(len(motifs)), 25)]
    for n, skel in enumerate(skels):
        fig, ax = plt.subplots()
        ax.axis("equal")
        ax.axis("off")

        for index in range(len(skel)):

            s = skel[index]

            s = translate(s, 12)
            rot_angle = angle(s[-1,0], s[-1,1], degrees = False)

            if rot_angle > np.pi/2:
                rotation = 3*(np.pi/2)-rot_angle
            else:
                rotation = -(np.pi/2+rot_angle)

            xs,ys = rotate_matrix(s, rotation)
            ax.plot(xs,ys, color = palette[index], alpha = 0.4)
            ax.set_title(cluster)

            fig.savefig(f"./prog_col_skels_{window}/progressive_skeleton_motifs_{cluster}_{window}_{n}.png")
            fig.savefig(f"./prog_col_skels_{window}progressive_skeleton_motifs_{cluster}_{window}_{n}.svg")
    

In [ ]:
len(palette)

In [ ]:
display(palette)

In [ ]:
random_selection

In [ ]:
import os

In [ ]:
for c in tqdm.tqdm(range(15)):
    try:
        make_animation(c)
    except Exception as e:
        print(c, str(e))